# Введение в обработку естественного языка
## Урок 3. Embedding word2vec fasttext
### Скачиваем датасет через ноутбук

In [14]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
!pip install apache_beam
!pip install scikit_learn
!pip install annoy
!pip install pymorphy2
!pip install stop_words

--2023-10-05 09:10:07--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2023-10-05 09:10:07--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc99bb67a8ddd87edb8be0ab569a.dl.dropboxusercontent.com/cd/0/inline/CFDtv_YXgK4wwla65zylHWPIIi3LPdADGbgE4kiMYXkK4ksBxsbA_pfFLsPuY0RIru2JwNuPbFhlBZj_iZrQpaSI70uqYnLEZXFQNvuqvL3Jtku0SyL7DdbPZTXizFC9dLM/file# [following]
--2023-10-05 09:10:07--  https://uc99bb67a8ddd87edb8be0ab569a.dl.dropboxusercontent.com/cd/0/inline/CFDtv_YXgK4wwla65zylHWPIIi3LPdADGbgE4kiMYXkK4ksBxsbA_pfFLsPuY0RIru2JwNuPbFhlBZj_iZrQpaSI70uqYnLEZXFQNvuqvL3Jtku0SyL7DdbP

In [15]:
import os
import urllib.request
from scipy import spatial
import os
import urllib.request
from scipy import spatial
import pandas as pd
import numpy as np
import codecs
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import re
from html.parser import HTMLParser
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from string import punctuation
from pymystem3 import Mystem
from gensim.models import Word2Vec, FastText
import annoy
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from string import punctuation
nltk.download('stopwords')
stop_words = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1. объединить в одну выборку

In [16]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = pd.concat([positive, negative])
df = df.reset_index().drop('index', axis=1)

In [17]:
df

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive
...,...,...
226829,Но не каждый хочет что то исправлять:( http://...,negative
226830,скучаю так :-( только @taaannyaaa вправляет мо...,negative
226831,"Вот и в школу, в говно это идти уже надо(",negative
226832,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative


### 2. на основе word2vec/fasttext/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)

In [18]:
def preprocess(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [19]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
for line in df['text'].values:
    spls = preprocess(line)
    sentences.append(spls)

In [22]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=3)
modelFT = FastText(sentences=sentences, vector_size=300, min_count=3, window=5, workers=8)

w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for line in df['text'].values:
    if len(line) > 0:
        n_w2v = 0
        n_ft = 0
        spls = line.split()

        question = preprocess(spls[0])

        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)

        counter += 1


w2v_index.build(10)
ft_index.build(10)

True

In [ ]:
def find_similar_word(emmbedes):
  nearest = sorted(emmbed_dict.keys(), key=lambda word: spatial.distance.euclidean(emmbed_dict[word], emmbedes))
  return nearest

### 3. Проверить насколько хорошо работают подходы

In [24]:
def get_response(question, index, model):
    question = preprocess(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    return index.get_nns_by_vector(vector, 5)

In [29]:
quest = 'снова на работу'

df.loc[get_response(quest, w2v_index, modelW2V)]

,text,label
160793,Работа кажется это реальная подстава от жизни....,negative
212515,Работа на конюшне убивает(((( @ КСК Московия h...,negative
220406,"Обратно в общагу. Не очень прикольная суббота,...",negative
12822,Суббота = спаааааааать ^___^ А потом на репети...,positive
13052,Суббота с любимым фильмом.хорошее было время т...,positive


In [30]:
df.loc[get_response(quest, ft_index, modelFT)]

,text,label
3373,Работа идет Игорь Сапко может пока писать в тв...,positive
3524,Работа и вкусный чайок :) @ Ведров http://t.co...,positive
11249,"Работа мне очень понравилось,думаю долго пораб...",positive
50491,"работа гребанное дно,а еще ведь домой пилить. ...",positive
63973,"Работа проходит в плановом режиме, дома предст...",positive


Выводы:

- word2vec и fasttext показали неплохие результаты и текст твитос соответствует содержимому запроса
.